In [1]:
import os,sys,gc,cv2,time
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import numpy as np
import pandas as pd 

from tqdm.notebook import tqdm
from tqdm.notebook import tqdm

from sklearn.metrics import roc_auc_score 
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import albumentations
import timm
from albumentations import *
from albumentations.pytorch import ToTensorV2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
model_path = ['../input/resnet200d-baseline-benchmark-public/resnet200d_fold0_cv953.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold1_cv955.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold2_cv955.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold3_cv957.pth',
              '../input/resnet200d-baseline-benchmark-public/resnet200d_fold4_cv954.pth']

LABELS = np.array(['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'])

N_LABELS = 11 
class CONFIG3:
    version = 1
    batchsize = 1
    imsize = 512 

In [3]:
class RANZCRResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d', out_dim=11, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, out_dim)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [4]:
transforms_test = albumentations.Compose([
    Resize(CONFIG3.imsize, CONFIG3.imsize),
    Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ToTensorV2()
])

## Metrics
def mean_roc_auc(targets,probabilities):
    roc_auc = [roc_auc_score(targets[:,k],probabilities[:,k]) for k in range(N_LABELS)]
    return np.average(roc_auc)

In [5]:
class RANZCRDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        self.labels = df[LABELS].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        img = cv2.imread(row.file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image']
        label = torch.tensor(self.labels[index]).float()
        if self.mode == 'test':
            return img
        else:
            return img, label

In [6]:
train = pd.read_csv('../input/ranzcr-sgkf-data/train_folds.csv')
train['file_path'] = train.StudyInstanceUID.apply(lambda x: os.path.join('../input/ranzcr-clip-catheter-line-classification/train', f'{x}.jpg'))

In [7]:
def tta_inference_func(model,test_loader):
    model.eval()
    bar = tqdm(test_loader)
    PREDS = []
    LOGITS = []

    with torch.no_grad():
        for batch_idx,images in enumerate(bar):
            x = images.to(device)
            x = torch.stack([x,x.flip(-1)],0) # hflip
            x = x.view(-1, 3, CONFIG3.imsize, CONFIG3.imsize)
            logits = model(x)
            logits = logits.view(CONFIG3.batchsize, 2, -1).mean(1)
            PREDS += [logits.sigmoid().detach().cpu()]
            LOGITS.append(logits.cpu())
        PREDS = torch.cat(PREDS).cpu().numpy()
        
    return PREDS

In [8]:
def cv_predict():
    scores = []
    for fold in range(5):
        print("-"*50)
        print(f"FOLD : {fold}")
        valid = train[train.fold == fold]
        valid_dset = RANZCRDataset(valid, 'test', transform=transforms_test)
        valid_loader = torch.utils.data.DataLoader(valid_dset, batch_size=CONFIG3.batchsize, shuffle=False,  num_workers=24)
        
        model = RANZCRResNet200D("resnet200d")
        model = model.to(device)
        model.load_state_dict(torch.load(model_path[fold], map_location='cuda:0'))
        pred = tta_inference_func(model,valid_loader)
        valid_labels = valid[LABELS].values
        auc = mean_roc_auc(valid_labels,pred)
        scores.append(auc)
        print(f"AUC : {auc}")
        
        pd.DataFrame(pred,columns=LABELS).to_csv(f"pred_torch_{CONFIG3.version}_{fold}.csv",index=False)
        del model,pred,valid,valid_dset
        gc.collect() 
    
    print(f"SCORE : {np.array(scores).mean()}")

In [9]:
cv_predict()

--------------------------------------------------
FOLD : 0


  0%|          | 0/6003 [00:00<?, ?it/s]

AUC : 0.955817257335
--------------------------------------------------
FOLD : 1


  0%|          | 0/6041 [00:00<?, ?it/s]

AUC : 0.9578174530536482
--------------------------------------------------
FOLD : 2


...

AUC : 0.957994366725293
--------------------------------------------------
FOLD : 3


  0%|          | 0/6018 [00:00<?, ?it/s]

AUC : 0.9598371456679544
--------------------------------------------------
FOLD : 4


  0%|          | 0/6030 [00:00<?, ?it/s]

AUC : 0.956302090659604
SCORE : 0.9575536626883
